# Soccer Formation Analysis and Visualisation

## Prerequisite Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

## Configurations

In [2]:
data_file_path = "data\\raw\\positions.log"

col_name =['frame_id', 'player_id','x_pos','y_pos']
length_of_field = 110.0
width_of_field = 72.0
dtype_dic = {'frame_id':np.int32,'player_id':np.int16,'x_pos':np.float64,'y_pos':np.float64}

## Preprocessing

In [3]:
data_df = pd.read_csv(data_file_path, delim_whitespace=True, header=None, names=col_name, dtype=dtype_dic)

In [4]:
data_df.head()

,frame_id,player_id,x_pos,y_pos
0,1,128,55.000000,36.000000
1,1,0,1.099998,36.000000
2,1,1,31.790001,53.628086
3,1,2,27.500000,41.352669
4,1,3,27.500000,29.449619


In [5]:
from bokeh.io import curdoc
from bokeh.layouts import row, widgetbox,column
from bokeh.models import ColumnDataSource, LabelSet, CustomJS, Title
from bokeh.models.widgets import Slider, Paragraph, Button, CheckboxButtonGroup
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [6]:
data_A = data_df[data_df['player_id'] < 128]
data_B = data_df[data_df['player_id'] > 128]

current_time = data_df['frame_id'].min()

source_df_A = data_A[data_A['frame_id']==current_time]
source_df_B = data_B[data_B['frame_id']==current_time]

In [7]:
source_df_A

,frame_id,player_id,x_pos,y_pos
1,1,0,1.099998,36.000000
2,1,1,31.790001,53.628086
3,1,2,27.500000,41.352669
4,1,3,27.500000,29.449619
5,1,4,31.790001,20.002220
6,1,5,55.000000,36.299999
7,1,6,40.283421,35.637085
8,1,7,55.000000,32.599998
9,1,8,49.939999,53.861343
10,1,9,52.711987,45.117290


In [9]:
def f(doc):
    source_coor_A = ColumnDataSource(data=dict(x=source_df_A['x_pos'], y=source_df_A['y_pos'], player_id = source_df_A['player_id']))
    source_coor_B = ColumnDataSource(data=dict(x=source_df_B['x_pos'], y=source_df_B['y_pos'], player_id = source_df_B['player_id'] ))
    
    plot = figure(name='base',plot_height=550, plot_width=850,
              title="Game Animation", tools="reset,save,wheel_zoom,pan",
              x_range=(0,110), y_range=(0,72), toolbar_location="below")
    
    plot.image_url(url=['background.png'], x=0, y=0, w=110, h=72, anchor='bottom_left')
    plot.scatter('x','y',source=source_coor_A, size=20)
    
    labels = LabelSet(x='x', y='y', text='player_id',
                  source=source_coor_A, y_offset=-8,
                  render_mode='canvas', text_color='black',
                  text_font_size="8pt", text_align='center')
    
    # https://github.com/samirak93/Game-Animation/blob/master/Animation/game_animation.py#L214
    """
    Remove plot background and alter other styles

    """
    def plot_clean(plot):

        plot.xgrid.grid_line_color = None
        plot.ygrid.grid_line_color = None
        plot.axis.major_label_text_font_size = "10pt"
        plot.axis.major_label_standoff = 0
        plot.border_fill_color = "white"
        plot.title.text_font = "times"
        plot.title.text_font_size = '10pt'
        plot.background_fill_color = "white"
        plot.title.align = 'center'
        return plot
    
    plot.add_layout(labels)
    plot.axis.visible = False
    plot = plot_clean(plot)
    
    slider_start = data_df.frame_id.min()
    slider_end = data_df.frame_id.max()
    game_time = Slider(title="Frame ID", value=slider_start,start=slider_start, end=slider_end, step=1)
    
    def update_data(attrname, old, new):
        slider_value = np.int32(game_time.value)

        source_df_A = data_A[data_A['frame_id']==slider_value]
        source_df_B = data_B[data_B['frame_id']==slider_value]

        source_coor_A.data = dict(x=source_df_A['x_pos'], y=source_df_A['y_pos'], player_id = source_df_A['player_id'])
        source_coor_B.data = dict(x=source_df_B['x_pos'], y=source_df_B['y_pos'], player_id = source_df_B['player_id'])    
    
    for w in [game_time]:
        w.on_change('value', update_data)
    
    layout = column(plot, game_time)
    
    doc.add_root(layout)
    
show(f, notebook_url='http://localhost:8889') # https://github.com/bokeh/bokeh/blob/master/examples/howto/server_embed/notebook_embed.ipynb

In [10]:
import xml.etree.ElementTree as ET

In [12]:
root = ET.parse('data\\raw\\Annotations_AtomicEvents_firstHalf.xml').getroot()

In [36]:
label_collected = ['BallPossession', 'BallOut', 'Goal', 'Foul']
bef_frame_id = 1
for track_node in root.findall('track'):
    try:
        if track_node.attrib['label'] in label_collected:
            box_node = track_node.find('box')
            if track_node.attrib['label'] == 'BallPossession':
                team_id_attrib = box_node.find("attribute[@name='teamId']")
                
    except:
        print(track_node.tag, track_node.attrib)
        print(box_node.tag, box_node.attrib)
        print(team_id_attrib.tag, team_id_attrib.text, team_id_attrib.attrib)
        raise

track {'id': '4407', 'label': 'Goal'}
box {'frame': '7651', 'keyframe': '1', 'occluded': '0', 'outside': '0', 'xbr': '1.24157999999999993e+03', 'xtl': '1.20052999999999997e+03', 'ybr': '1.68740000000000009e+02', 'ytl': '9.17900000000000063e+01'}


AttributeError: 'NoneType' object has no attribute 'tag'